In [1]:
!nvidia-smi

Mon Jul  5 22:23:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 461.33       Driver Version: 461.33       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 1650   WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   44C    P8     3W /  N/A |    624MiB /  4096MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
import torch
import os
import pandas as pd

from deepmatcher.data.custom_dataset import DeepMatcherDataset
from deepmatcher.data.encoder.text_encoders import HFTextEncoder

from torch.utils.data import DataLoader
from deepmatcher.models.core import MatchingModel

In [2]:
dataset_dir = "examples/sample_data/itunes-amazon/"
train_df = pd.read_csv(os.path.join(dataset_dir, 'train.csv'))
print('train:', train_df.shape)
val_df = pd.read_csv(os.path.join(dataset_dir, 'validation.csv'))
print('val:', val_df.shape)
test_df = pd.read_csv(os.path.join(dataset_dir, 'train.csv'))
print('test:', test_df.shape)

train_df.head()

train: (323, 18)
val: (108, 18)
test: (323, 18)


,id,label,left_Song_Name,left_Artist_Name,left_Album_Name,left_Genre,left_Price,left_CopyRight,left_Time,left_Released,right_Song_Name,right_Artist_Name,right_Album_Name,right_Genre,right_Price,right_CopyRight,right_Time,right_Released
0,448,0,Baby When the Light ( David Guetta & Fred Rist...,David Guetta,Pop Life ( Extended Version ) [ Bonus Version ],"Dance , Music , Rock , Pop , House , Electroni...",$ 1.29,‰ ãÑ 2007 Gum Records,6:17,18-Sep-07,Revolver ( Madonna Vs. David Guetta Feat . Lil...,David Guetta,One Love ( Deluxe Version ),Dance & Electronic,$ 1.29,( C ) 2014 Swedish House Mafia Holdings Ltd ( ...,3:18,"August 21 , 2009"
1,287,1,Outversion,Mark Ronson,Version,"Pop , Music , R&B / Soul,Soul,Dance,Rock,Jazz,...",$ 0.99,2007 Mark Ronson under exclusive license to SO...,1:50,10-Jul-07,Outversion,Mark Ronson,Version [ Explicit ],Pop,$ 0.99,( c ) 2011 J'adore Records,1:50,"July 10 , 2007"
2,534,0,Peer Pressure ( feat . Traci Nelson ),Snoop Dogg,Doggumentary,"Hip-Hop/Rap , Music , Rock , Gangsta Rap , Wes...",$ 1.29,"‰ ãÑ 2011 Capitol Records , LLC . All rights r...",4:07,29-Mar-11,Boom ( ( Feat . T-Pain ) [ Edited ] ),Snoop Dogg,Doggumentary [ Edited ],"Rap & Hip-Hop , West Coast",$ 1.29,"( C ) 2011 Capitol Records , LLC",3:50,"March 29 , 2011"
3,181,1,Stars Come Out ( Tim Mason Remix ),Zedd,Stars Come Out ( Remixes ) - EP,"Dance , Music , Electronic , House",$ 1.29,2012 Dim Mak Inc.,5:49,20-May-14,Stars Come Out ( Dillon Francis Remix ),Zedd,Stars Come Out [ Dillon Francis Remix ],Dance & Electronic,$ 1.29,2012 Dim Mak Inc.,4:08,"May 20 , 2014"
4,485,0,Jump ( feat . Nelly Furtado ),Flo Rida,R.O.O.T.S. ( Deluxe Version ),"Hip-Hop/Rap , Music",$ 1.29,‰ ãÑ 2009 Atlantic Recording Corporation for t...,3:28,30-Mar-09,"Yayo [ Feat . Brisco , Billy Blue , Ball Greez...",Flo Rida,R.O.O.T.S. ( Route Of Overcoming The Struggle ...,Rap & Hip-Hop,$ 1.29,"( C ) 2012 Motown Records , a Division of UMG ...",7:53,"March 30 , 2009"


In [3]:
text_transformer = HFTextEncoder('distilbert-base-uncased', max_length=16, truncation=True, padding='max_length', trainable=True)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
label_column = 'label'
text_columns = ['Song_Name', 'Artist_Name', 'Album_Name', 'Genre', 'Price', 'CopyRight', 'Time', 'Released']
image_col = ''
tokenizer = text_transformer.tokenizer

train_dataset = DeepMatcherDataset(
    data_df=train_df,
    label_col=label_column,
    text_cols=text_columns,
    image_col=image_col,
    tokenizer=tokenizer
)

val_dataset = DeepMatcherDataset(
    data_df=val_df,
    label_col=label_column,
    text_cols=text_columns,
    image_col=image_col,
    tokenizer=tokenizer
)

test_dataset = DeepMatcherDataset(
    data_df=test_df,
    label_col=label_column,
    text_cols=text_columns,
    image_col=image_col,
    tokenizer=tokenizer
)


received data_df : (323, 18)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 323 entries, 0 to 322
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 323 non-null    int64 
 1   label              323 non-null    int64 
 2   left_Song_Name     323 non-null    object
 3   left_Artist_Name   323 non-null    object
 4   left_Album_Name    323 non-null    object
 5   left_Genre         323 non-null    object
 6   left_Price         323 non-null    object
 7   left_CopyRight     323 non-null    object
 8   left_Time          323 non-null    object
 9   left_Released      312 non-null    object
 10  right_Song_Name    323 non-null    object
 11  right_Artist_Name  323 non-null    object
 12  right_Album_Name   323 non-null    object
 13  right_Genre        323 non-null    object
 14  right_Price        323 non-null    object
 15  right_CopyRight    323 non-null    object
 16  right_Time     

In [5]:
for batch in train_dataset:
    break
batch

{'attrs': defaultdict(dict,
             {'Song_Name': {'left_': {'input_ids': tensor([  101,  3336,  2043,  1996,  2422,  1006,  2585, 19739, 16549,  1004,
                         5965, 15544,  6238,  6136,  1007,   102]),
                'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])},
               'right_': {'input_ids': tensor([  101, 17863,  1006, 11284,  5443,  1012,  2585, 19739, 16549,  8658,
                         1012, 13451,  1005,  6159,  1007,   102]),
                'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}},
              'Artist_Name': {'left_': {'input_ids': tensor([  101,  2585, 19739, 16549,   102,     0,     0,     0,     0,     0,
                            0,     0,     0,     0,     0,     0]),
                'attention_mask': tensor([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])},
               'right_': {'input_ids': tensor([  101,  2585, 19739, 16549,   102,     0,     0,     0,     0,  

In [6]:
batch['attrs']['Song_Name']['left_']['input_ids'].shape

torch.Size([16])

In [7]:
model = MatchingModel(
    text_encoder=text_transformer,
    attr_summarizer=None,
    attr_condense_factor='auto',
    attr_comparator='concat',
    attr_merge='concat',
    classifier='2-layer-highway',
    hidden_size=300
    )

In [8]:
model

MatchingModel(
  (text_encoder): HFTextEncoder(
    (model): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0): TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768, out_features=768, bias=True)
            )
            (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (ffn): FFN(
              (d

In [9]:
init_batch = next(iter(DataLoader(train_dataset, batch_size=2)))['attrs'] ## draw a batch to initialize the lazy model 
model.initialize(train_dataset, init_batch=init_batch)
model

D:\installations\anaconda\envs\aisle3-torch\lib\site-packages\torch\nn\modules\module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


MatchingModel(
  (text_encoder): HFTextEncoder(
    (model): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0): TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768, out_features=768, bias=True)
            )
            (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (ffn): FFN(
              (d

In [10]:
model.run_train(
    train_dataset,
    val_dataset,
    best_save_path='best_mode.pth',
    batch_size=16,
    epochs=5,
    )

* Number of trainable parameters: 67639882
===>  TRAIN Epoch 1


D:\installations\anaconda\envs\aisle3-torch\lib\site-packages\torch\nn\functional.py:2741: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(
0% [████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:07


Finished Epoch 1 || Run Time:    6.9 | Load Time:    0.3 || F1:   2.38 | Prec:  14.29 | Rec:   1.30 || Ex/s:  44.69

===>  EVAL Epoch 1


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


Finished Epoch 1 || Run Time:    1.6 | Load Time:    0.1 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:  61.96

* Best F1: tensor(0., device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


Finished Epoch 5 || Run Time:    5.0 | Load Time:    0.3 || F1:  74.48 | Prec:  79.41 | Rec:  70.13 || Ex/s:  61.19

===>  EVAL Epoch 5


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


Finished Epoch 5 || Run Time:    1.6 | Load Time:    0.1 || F1:  50.00 | Prec:  75.00 | Rec:  37.50 || Ex/s:  61.86

---------------------

Loading best model...
Training done.


tensor(62.2222, device='cuda:0')

In [11]:
model.run_eval(test_dataset)

===>  EVAL Epoch 3
/home/sour4bh/.local/lib/python3.8/site-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
0% [██] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04
Finished Epoch 3 || Run Time:    4.7 | Load Time:    0.6 || F1:  77.71 | Prec:  76.25 | Rec:  79.22 || Ex/s:  61.52



tensor(77.7070, device='cuda:0')